Code scaffolding for classifying Mitral Annular Calcification using echocardiography images.

In [1]:
# Import relevant libraries

import os
import pickle
import gzip
import numpy as np
from PIL import Image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import decode_predictions
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
import matplotlib.pyplot as plt
from keras.applications import inception_v3
%matplotlib inline


Using TensorFlow backend.


In [7]:
# Opening and displaying sample images

# Designate file path
filename = 'filename.pkl.gz'

# Unzip and unpickle file, creates a numpy array of grayscale pixel values
with gzip.open(filename, 'rb') as f1:
    u = pickle.Unpickler(f1, encoding='bytes').load()
    
# Select an aribitrary slide number and reshapes to 800 x 600 matrix for display
image = np.reshape(u[24],(600,800))

# Display images as grayscale
img = Image.fromarray(image, 'L')
img.show()

In [8]:
# Predicting image class based on pretrained inception network 

# Load inception model
inception_model = inception_v3.InceptionV3(weights='imagenet')

# Convert from single channel image (grayscale), to three channel image (still grayscale but with three channels)
image_3channel = np.dstack([image]*3)
 
# Add an additional dimension at position 0 to signify batch number (inception requires a four dimension vector)
image_4D = np.expand_dims(image_3channel, axis=0)

# prepare the image for inception
processed_image = inception_v3.preprocess_input(image_4D.copy())
 
# get the predicted probabilities for each class
predictions = inception_model.predict(processed_image)

# print predictions
label = decode_predictions(predictions)
print(label)


[[('n06359193', 'web_site', 0.009563069), ('n04286575', 'spotlight', 0.0065397914), ('n01930112', 'nematode', 0.00578471), ('n04141076', 'sax', 0.0051469584), ('n03759954', 'microphone', 0.004586166)]]


In [13]:
# Customize inception by adding layers and changing output to 4 classes (absent, mild, moderate, severe)

# create the base pre-trained model
base_model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add a dropout layer
x = Dropout(0.5)(x)

# Add a logistic layer
predictions = Dense(4, activation='softmax')(x)

# Final model to train
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all but the last seven layers of the network for training
for layer in model.layers[:-7]:
    layer.trainable = False

# Compile network to incorporate changes
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Display model architecture
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_352 (Conv2D)             (None, None, None, 3 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_352 (BatchN (None, None, None, 3 96          conv2d_352[0][0]                 
__________________________________________________________________________________________________
activation_351 (Activation)     (None, None, None, 3 0           batch_normalization_352[0][0]    
__________________________________________________________________________________________________
conv2d_353

References:
- https://www.learnopencv.com/keras-tutorial-using-pre-trained-imagenet-models/ (v3 model stuff)
- https://blog.coast.ai/five-video-classification-methods-implemented-in-keras-and-tensorflow-99cad29cc0b5
